<a href="https://colab.research.google.com/github/Adam-Rao/Cats-and-Dogs/blob/model/CatsAndDogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!unzip "/content/drive/My Drive/kagglecatsanddogs_3367a.zip" -d "/content/drive/My Drive/Cats and Dogs Project"

In [ ]:
import _io
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle as pkl
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import tensorflow_hub as hub
from typing import List, Tuple

In [ ]:
BATCH_SIZE = 32
DATA_LIST_TYPE = List[Tuple[str, str]]
IMG_SIZE = 224
INPUT_SHAPE = [None, IMG_SIZE, IMG_SIZE, 3]
FINAL_LAYER_ACTIVATION_FUNCTION = "sigmoid"
OUTPUT_NODES = 2
NUM_EPOCHS=100

In [ ]:
def create_data_list(data_path: str , label: str) -> DATA_LIST_TYPE:
  """
  data_path: path to folder containing desired files
  label: string argument for the desired label

  Creates and returns a list of tuples containing filepath and desired label
  """
  dataset: DATA_LIST_TYPE = []
  for file in os.listdir(data_path):
    filename, file_ext = os.path.splitext(file)
    if file_ext == ".jpg":
      dataset.append((f"{data_path}/{file}", label))

  return dataset

In [ ]:
cat_data_list = create_data_list("/content/drive/My Drive/Cats and Dogs Project/PetImages/Cat", "CAT")
dog_data_list = create_data_list("/content/drive/My Drive/Cats and Dogs Project/PetImages/Dog", "DOG")

general_data_list = cat_data_list + dog_data_list

result = is_image("/content/drive/My Drive/Cats and Dogs Project/PetImages/Cat/95.jpg", verbose=False)
print(result)

None


In [ ]:
def shuffle_data_list(data_list: DATA_LIST_TYPE) -> DATA_LIST_TYPE:
  """
  data_list: List of tuples holding data
  Returns shuffled list of tuples
  """
  return shuffle(data_list)

In [ ]:
shuffled_data_list = shuffle_data_list(general_data_list)

In [ ]:
def save_data_list(data_list: DATA_LIST_TYPE, file_path: str) -> None:
  """
  data_list: List of tuples to be saved
  file_path: Path to file where list is to be saved

  Saves data list to specified file
  """

  save_file = open(file_path, "wb")
  pkl.dump(data_list, save_file)

In [ ]:
def load_data_file(file_path: str) -> _io.BufferedReader:
  """
  file_path: Path to file containing saved data
  Loads and returns saved file
  """
  return open(file_path, "rb")

In [ ]:
save_data_list(shuffled_data_list, "/content/drive/My Drive/Cats and Dogs Project/Extracted Data/data.pkl")

In [ ]:
with load_data_file("/content/drive/My Drive/Cats and Dogs Project/Extracted Data/data.pkl") as f:
  shuffled_data = pkl.load(f)

In [ ]:
labels = [i[1] for i in shuffled_data]

unique_labels = np.unique(labels)

In [ ]:
boolean_labels = [label == unique_labels for label in labels] # will form our y values

In [ ]:
def create_train_test_datasets(X, y, TEST_SIZE=0.2, RANDOM_STATE=42):
  """
  X: Feature values
  y: Label values
  TEST_SIZE: Percentage of test dataset from original. Default is 20%
  RANDOM_STATE: Random integer. Default is 42

  Creates train and test datasets from X and y values.
  Can also be used to create train and validation datasets
  """
  X_train, X_test, y_train, y_test = train_test_split(
      X, 
      y, 
      test_size=TEST_SIZE, 
      random_state=RANDOM_STATE
    )
  
  return X_train, X_test, y_train, y_test

In [ ]:
def preprocess_image(file_path: str):
  """
  file_path: File to image
  Returns preprocessed image
  """

  image = tf.io.read_file(file_path)
  image = tf.image.decode_jpeg(image)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])

  return image

In [ ]:
def return_image_label(file_path: str, label: str):
  """
  file_path: Path to image
  label: Image label

  returns preprocessed image and label
  """
  image = preprocess_image(file_path)

  return image, label

In [ ]:
def create_data_batches(
    X,
    y=None,
    test_data=False,
    valid_data=False
  ):
  """
  X: Feature values(Images)
  y: Label values
  test_data: Boolean value determining if to create data batches for test data
  valid_data: Boolean value determining if to create data batches for validation data

  Creates and returns data batches from X and/or y values
  """

  if test_data:
    print("Creating test data batches...")
    data = tf.data.Dataset.from_tensor_slices(tf.constant(X))
    data_batch = data.map(preprocess_image).batch(BATCH_SIZE)
    return data_batch

  elif valid_data:
    print("Creating validation data batches...")
    data = tf.data.Dataset.from_tensor_slices(
        (
            tf.constant(X),
            tf.constant(y)
        )
    )
    data_batch = data.map(return_image_label).batch(BATCH_SIZE)
    return data_batch
  
  else:
    print("Creating training data batches...")
    data = tf.data.Dataset.from_tensor_slices(
        (
            tf.constant(X),
            tf.constant(y)
        )
    )
    data = data.shuffle(buffer_size=len(X))
    data_batch = data.map(return_image_label).batch(BATCH_SIZE)
    return data_batch

In [ ]:
def get_prediction_label(prediction_probabilities):
  """
  prediction_probabilities: Array of predictions from model

  returns predicted label
  """

  return unique_labels[np.argmax(prediction_probabilities)]

In [ ]:
def plot_image_prediction_true_value(prediction_probabilities, X, y, index):
  """
  prediction_probabilities: Array of predictions from model
  X: image to be plotted
  y: true label
  index: Position of image, prediction and true label you wish to show

  plot actual image with the predicted value and true values as title to image plot
  """
  image, true_label, prediction_probabilities = X[index], y[index], predicition_probabilites[index]
  predicted_label = get_prediction_label(prediction_probabilities)

  plt.imshow(image)
  plt.yticks([])
  plt.xticks([])
  plt.title(
      f"Predicted Label: {predicted_label}  True Label: {true_label}\n" 
      f"Probability image is predicted label: {np.argmax(prediction_probabilities) * 100:.2f}")
  

In [ ]:
def unbatchify(batched_dataset):
  """
  returns list of images and lables from unbatched dataset
  """
  images_ = []
  labels_ = []

  for image, label in batched_dataset.unbatch().as_numpy_iterator():
    images_.append(image)
    labels_.append(unique_labels[np.argmax(label)])

  return images_, labels_

In [ ]:
def create_model():
  """
  Creates and returns custom model: Model based on INCEPTION Architecture
  """
  model = Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v1/classification/4"),
    Dense(
      OUTPUT_NODES, 
      activation=FINAL_LAYER_ACTIVATION_FUNCTION)
  ])

  model.compile(
      optimizer='adam',
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=['accuracy']
  )

  model.build(INPUT_SHAPE)

  model.summary()
  
  return model

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=3
  )

In [ ]:
def save_model(model, suffix=None):
  modeldir = os.path.join(
      'drive/My Drive/Cats and Dogs Project/models',
      datetime.datetime.now().strftime("%Y%m%d-%H%M%s")
  )
  model_path = modeldir + "-" + suffix + ".h5"
  print(f"Saving model to: {model_path}...")
  model.save(model_path)
  return model_path

In [ ]:
def load_model(model_path):
  print(f"Loading saved model from: {model_path}")
  model = tf.keras.models.load_model(
      model_path,
      custom_objects={"KerasLayer":hub.KerasLayer}
  )
  return model

In [ ]:
X = [i[0] for i in shuffled_data]

trial_X = X[:1000]
trial_y = boolean_labels[:1000]


trial_X_train, trial_X_valid, trial_y_train, trial_y_valid = create_train_test_datasets(trial_X, trial_y)

trial_train_data = create_data_batches(
    trial_X_train,
    trial_y_train
)

trial_valid_data = create_data_batches(
    trial_X_valid,
    trial_y_valid,
    valid_data=True
)

Creating training data batches...
Creating validation data batches...


In [ ]:
model = create_model()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 1001)              6633209   
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 2004      
Total params: 6,635,213
Trainable params: 2,004
Non-trainable params: 6,633,209
_________________________________________________________________


In [ ]:
model.fit(
    x=trial_train_data,
    epochs=NUM_EPOCHS,
    validation_data=trial_valid_data,
    validation_freq=1,
    callbacks=[early_stopping]
)

Epoch 1/100
 6/25 [======>.......................] - ETA: 4:08 - loss: 0.2294 - accuracy: 0.9635

InvalidArgumentError: ignored

In [ ]:
while True: pass #keeps codelab from disconnecting